# Control points optimization - differential evolution

## Loop

In [1]:
from shapeOptInductor import gen_meshN, Id
import ngsolve as ngs
from ngsolve.webgui import Draw
import numpy as np
import matplotlib.pyplot as plt

## 1 - Geometry and meshing

We define a geometry with an arbitrary number of control points on the airgap.

In [2]:
NControlPoints = 10
airgap = [1e-3,2e-3]*(NControlPoints//2)
lz = 1e-2  # thickness in the z-direction (m)
s = 4  # symmetry factor

maxh = 4e-3
mesh, _, _ = gen_meshN(airgap, maxh)

XiAir = mesh.MaterialCF({"air": 1})
XiCore = mesh.MaterialCF({"core": 1})
XiCoil = mesh.MaterialCF({"coil": 1})

materialFunction = 1 * XiAir + 2 * XiCoil + 3 * XiCore
Draw(materialFunction, mesh, radius=0.02)


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

## 2 - State problem

### a) Parameters definition

In [3]:
f = 5e4  # working frequency (Hz)
omega = 2 * np.pi * f  # rad/s
mu0 = 4e-7 * np.pi  # void permeability (H/m)
mur = 1000  # relative permeability of iron (no unit)
mu_iron = mur * mu0  # permeability of iron (H/m)
delta = 0.1  # loss angle associated with the coil (rad)
mu_coil = np.exp(-1j * delta) * mu0  # complex permeability
nb_turn = 200  # number of turn in the coil (no unit)
Is = 2  # source current intensity (A)
js = nb_turn / 2 * Is / (ngs.Integrate(XiCoil, mesh)) * XiCoil  # source current density (A/m²)


### b) Definition of the magnetic problem

In [4]:
def magWeakFormComplex(a, a_):
    """Return the complex weak form of the magnetic problem, i.e, bilinear
    and linear forms (matrix and right-hand side after discretization, respectively)"""

    # bilinear form (transmission)
    bf = ngs.grad(a_) * 1 / mu_iron * ngs.grad(a) * ngs.dx("core")
    bf += ngs.grad(a_) * 1 / mu_coil * ngs.grad(a) * ngs.dx("coil")
    bf += ngs.grad(a_) * 1 / mu0 * ngs.grad(a) * ngs.dx("air")

    # linear form (source)
    lf = a_ * js * ngs.dx("coil")

    return bf, lf


def solveStateComplex(mesh):
    """Solve the complex magnetic state"""
    # definition of function space
    fes = ngs.H1(mesh, order=1, dirichlet="arc|segment2|domainVert", complex=True)
    a, a_ = fes.TnT()

    # definition of weak form
    bf, f = magWeakFormComplex(a, a_)
    K, F = ngs.BilinearForm(fes), ngs.LinearForm(fes)
    K += bf
    F += f

    # assembly
    K.Assemble()
    F.Assemble()

    # solving
    gf = ngs.GridFunction(fes)
    Kinv = K.mat.Inverse(freedofs=fes.FreeDofs(), inverse="pardiso")
    gf.vec.data = Kinv * F.vec

    return gf


### c) Definition of the post-processed quantities

We define the losses and inductance. We don't need derivatives here since we'll use a differential evolution algorithm, which is a gradient-free optimization algorithm.

In [5]:
def Inductance(a, mesh):
    rel = XiAir / mu0 + XiCoil / mu_coil + XiCore / mu_iron
    return s * lz / (Is**2) * ngs.Integrate(rel.real * ngs.Norm(ngs.grad(a)) ** 2, mesh)


def Losses(a, mesh):
    rel = XiCoil / mu_coil
    return s * np.pi * f * lz * ngs.Integrate(rel.imag * ngs.Norm(ngs.grad(a)) ** 2, mesh)


## 3 - Optimization

### a) Scipy wrapper
We first define wrappers of the objective function to minimize (the losses) and the constraint (the inductance value). We have to set a tolerance for the equality constraint, that we chose to be $\pm5\%$.

In [6]:
# FEM evaluation counter (solve the FEM system)
FEMcounter = [0]
LossList = []
xList = []


def scipyObj(x, maxh = 4e-3):
    """ Scipy wrapper to the objective function to minimize """
    FEMcounter[0] += 1                      # Increment at each evaluation
    if FEMcounter[0] % 100 == 0:            # print every 100 function calls
        print(f"{FEMcounter[0]} function evaluations")
    mesh, _, _ = gen_meshN(list(x), maxh)
    a = solveStateComplex(mesh)
    return Losses(a, mesh)

def scipyConst(x, maxh = 4e-3):
    """ Scipy wrapper to the constraint to respect """
    FEMcounter[0] += 1                      # Increment at each evaluation
    if FEMcounter[0] % 100 == 0:            # print every 100 function calls
        print(f"Evaluation {FEMcounter[0]}")
    mesh, _, _ = gen_meshN(list(x), maxh)
    a = solveStateComplex(mesh)
    return Inductance(a, mesh) - 1e-3

def callback(intermediate_result):
    LossList.append(intermediate_result.fun)
    xList.append(intermediate_result.x)


from scipy.optimize import NonlinearConstraint, Bounds
nlc = NonlinearConstraint(scipyConst,-5e-5,5e-5)


### b) Optimization

We use `differential_evolution` from `scipy.optimize` package.

In [ ]:
from scipy.optimize import differential_evolution
import pickle

NList = [4,6,8,10,12,14,16,18,20]

FEMcounterListAll = []
objCounterListAll = []
LossListAll = []
xListAll = []

for N in NList:

    FEMcounter = [0]
    LossList = []
    xList = []
    bnds = Bounds(5e-4*np.ones(N), 7.2e-3*np.ones(N))

    print("-------------------------------")
    print("Number of control points : ", N)

    result = differential_evolution(
    scipyObj,
    bounds=bnds,
    maxiter=100,
    popsize=15,
    tol = 0.1,
    atol= 0.1,
    seed=0, # for reproducibility
    disp=True,
    callback=callback,
    polish=False,
    constraints = nlc)

    c = FEMcounter[0]
    FEMcounterListAll.append(c)
    xListAll.append(xList.copy())
    LossListAll.append(LossList.copy())
    objCounterListAll.append(result.nfev)

    # Saving the results
    filename = "results/differential_evolution_N" + str(N) + ".pkl"
    mesh, _, _ = gen_meshN(result.x, maxh = 4e-3)
    a = solveStateComplex(mesh)
    Lopt = Inductance(a,mesh)
    results = {"xList" : xList,
               "LossList" : LossList,
               "FEMcounter" : FEMcounter[0],
               "objCounter" : result.nfev,
               "xOpt" : result.x,
               "lossOpt" : result.fun,
               "inductanceOpt" : Lopt,
               "mesh" : mesh}
    with open(filename, 'wb') as fileToSave:
        pickle.dump(results, fileToSave)

-------------------------------
Number of control points :  4
Evaluation 100
differential_evolution step 1: f(x)= 10.161997442272973
Evaluation 200
differential_evolution step 2: f(x)= 8.98919896715396


## 4) Results

### a) Design

In [ ]:
meshOpt, _, _ = gen_meshN(result.x)
Draw(materialFunction, meshOpt, radius=0.02)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

### b) Performance

In [ ]:
with open(r"results/differential_evolution_N10.pkl", "rb") as input_file:
    resultN10 = pickle.load(input_file)

In [ ]:
resultN10

{'xList': [], 'LossList': [], 'FEMcounter': 11340, 'objCounter': 2037}

In [ ]:
InductanceListOptim = []
LossListOptim = []
for xlist in xListAll:
    meshOpt, _, _  = gen_meshN(xlist[-1])
    a = solveStateComplex(meshOpt)
    InductanceListOptim.append(Inductance(a, meshOpt))
    LossListOptim.append(Losses(a, meshOpt))

ax1 = plt.plot(NList, LossListOptim, color="b")
plt.ylabel("Losses (W)", color="b")
plt.xlabel("Number of control points")
ax2 = plt.gca().twinx()
ax2.plot(NList, InductanceListOptim, color="r")
plt.ylabel("Inductance (H)", color="r")
plt.show()



IndexError: list index out of range

In [ ]:
plt.plot(NList, FEMcounterListAll)
plt.title("FEM counter")
plt.xlabel("Number of control points")

## Saving the results

In [ ]:
import pickle
results = dict("xListAll" : xListAll,"LossListAll" : LossListAll, "FEMcounterListAll" : FEMcounterListAll, "objCounterListAll" : objCounterListAll )
with open('results/differential_evolution.pkl', 'wb') as f:
    pickle.dump(results, f)

In [ ]:
filename = "results/test" + str(N) + ".pkl"
results = {"xOpt" : result.x, "N" : N, "mesh" : mesh }
with open(filename, 'wb') as fileToSave:
    pickle.dump(results, fileToSave)

In [ ]:
with open(filename, "rb") as input_file:
    resultTest = pickle.load(input_file)

In [ ]:
resultTest

{'xOpt': array([0.00544366, 0.00489403, 0.00059767, 0.00061943, 0.00425711,
        0.00530268, 0.00560901, 0.00716929, 0.00647654, 0.00688958,
        0.00674568, 0.00634516, 0.00535751, 0.00603298, 0.00610666,
        0.00352439, 0.00683643, 0.00105619, 0.00578528, 0.00336295]),
 'N': 20,
 'mesh': <ngsolve.comp.Mesh at 0x2847b2da6f0>}

In [ ]:
result

             message: Optimization terminated successfully.
             success: True
                 fun: 4.359892736379457
                   x: [ 5.444e-03  4.894e-03 ...  5.785e-03  3.363e-03]
                 nit: 35
                nfev: 2522
          population: [[ 5.444e-03  4.894e-03 ...  5.785e-03  3.363e-03]
                       [ 2.109e-03  6.900e-03 ...  5.594e-03  6.923e-03]
                       ...
                       [ 1.060e-03  2.804e-03 ...  7.044e-03  6.997e-03]
                       [ 5.123e-04  4.633e-03 ...  5.261e-03  5.231e-03]]
 population_energies: [ 4.360e+00  6.017e+00 ...  6.167e+00  6.359e+00]
              constr: [array([ 0.000e+00])]
    constr_violation: 0.0
               maxcv: 0.0